In [ ]:
"""
Python stores strings in unicode, they are not simply a vec of bytes.
A stream/vec of bytes in python is a string prepended with b.
"""
print([i for i in 'abc'])
print([i for i in b'abc'])

In [ ]:
"""
Unfortunately the display will automatically show relevant bytes as 
ASCII characters, but they are just bytes, not strings.
"""
import os

os.urandom(5)

In [ ]:
print('abc'.encode('utf16'))
print(b'\xff\xfea\x00b\x00c\x00'.decode('utf16'))

In [ ]:
import dis

def f(): x = 1
print(f.__code__.co_code)
print()
dis.disassemble(f.__code__)

In [ ]:
# Server - Run this here and then run the client section separately.
# Can also be done with netcat.
import socket
server = socket.socket()
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Release the server on quit
server.bind(('0.0.0.0', 8000))  # Listen on port 8000 from any source IP (?)
server.listen(1000)  # Listen to up to 1000 connections
client, address = server.accept()  # Hang until a connection comes through
print(client.getpeername())
print(client.getsockname())
print(client.recv(1024))
client.sendall(b'Hey there.')
client.close()
server.close()

In [ ]:

# Client - Run in parallel to Server section (REPL).
# Can also be done with netcat.
import socket
conn = socket.socket()
conn.connect(('127.0.0.1', 8000))  # Listen on port 8000 from any source (any source NIC?)
conn.getpeername()
conn.getsockname()
conn.sendall(b'Hello!')
conn.recv(1024)
conn.close()

In [ ]:
# Simple HTTP server
# Can run from command line:
#   curl "127.0.0.1:8000/hello"
import http.server

class Handler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path != '/hello':
            self.send_response(404)
            self.end_headers()
            return

        data = b'Hello, world!'
        self.send_response(200)
        self.send_header('Content-Type', 'text/plain')
        self.send_header('Content-Length', len(data))
        self.end_headers()
        self.wfile.write(data)

http_server = http.server.HTTPServer(('0.0.0.0', 8001), Handler)
http_server.serve_forever()

In [1]:
# Multiplethreaded echo server.
import threading
import socket

# Creating a new handling creates a new thread which will 
# run the relevant code.
class ThreadHandler(threading.Thread):
    def __init__(self, connection):
        super().__init__()
        self.connection = connection
        self.lock = threading.Lock()
    # calling Thread.start triggers `run` to run in a separate 
    # thread.
    def run(self):
        while True:
            data = self.connection.recv(1024)
            if not data:
                break
            # Perform the critical section in a single threaded manner.
            # This will block other Handlers from entering their own
            # critical sections, but not the Server from spawning
            # new threads.
            with self.lock:
                # Imagine a race sensitive activity, such as writing
                # to a file.
                self.connection.sendall(data)

# Multiplethreaded echo server.
class Server:
    def __init__(self, host, port) -> None:
        self.host = host
        self.port = port
    def start(self):
        listener = socket.socket()
        listener.bind((self.host, self.port))
        listener.listen(1000)
        while True:
            connection, address = listener.accept()
            # Create a handler which will handle this connection
            # in a separate thread.
            handler = ThreadHandler(connection)
            handler.start()

server = Server('127.0.0.1', 8002)
server.start()

OSError: [Errno 98] Address already in use